In [14]:
# import pandas as pd
import csv
import os, re, sys
from collections import Counter
from IPython.core.debugger import Tracer; debug_here = Tracer()
from nltk import word_tokenize

In [3]:
# Talk page usernames
csvpath = '/home/michael/school/research/wp/wikipedia/data/talk/ipc_utf8_talkpages.csv'
talk_counter = Counter()

with open(csvpath, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        talk_counter[row[3]] += 1

talk_eds = set(talk_counter.keys())
print(len(talk_eds))

5537


In [90]:
preprocess("<ref name='")

''

In [91]:
preprocess('<ref name="something"')

''

In [92]:
preprocess('all these thing <ref name="something">')

'all these thing '

In [93]:
preprocess('<ref name="something"> or nothing')

' or nothing'

In [103]:
preprocess('{{cite web|url=|title=Peace talks set to begin after Israel agrees to free 104 Palestinian prisoners}}')

''

In [106]:
# Process text

#urlpat = re.compile(r'https?:\/\/(?:www\.)?[^ ]*')
#urlhtml = re.compile(r'[^ ].*?\.html?')
#temppat = re.compile(r'\{\{.*?(?:\}\}|$)')
#tempout = re.compile(r'^.*?\}\}')
#linkpat = re.compile(r'\[\[.*?\|')

pats = []
# urls
pats.append(r'https?:\/\/(?:www\.)?[^ ]*')
pats.append(r'[^ ].*?\.html?')

# templates
pats.append(r'\{\{.*?(?:\}\}|$)')
pats.append(r'^.*?\}\}')

# links
pats.append(r'\[\[.*?\|')

# refs
# pats.append(r'<ref name=.*?[>$]')
pats.append(r'<ref name=.*?(?:>|$)')
pat = re.compile('|'.join(pats))

def preprocess(intext):
    outtext = pat.sub('', intext)
    # urls
#    outtext = re.sub(r'https?:\/\/(?:www\.)?[^ ]*', '', intext)
##     outtext = re.sub(r'https?:\/\/(?:www\.)?[^ ]*', '', intext, flags=re.MULTILINE)
##     outtext = urlpat.sub('', intext)
##     print('1 sub')
##     sys.stdout.flush()
#    outtext = re.sub(r'[^ ].*?\.html?', '', outtext)
##     outtext = re.sub(r'[^ ].*?\.html?', '', outtext, flags=re.MULTILINE)
##     outtext = urlhtml.sub('', outtext)
##     print('2 sub')
##     sys.stdout.flush()
#    
#    # templates
#    outtext = re.sub(r'\{\{.*?(?:\}\}|$)','', outtext)
##     outtext = re.sub(r'\{\{.*?(?:\}\}|$)','', outtext, flags=re.MULTILINE)
##     outtext = temppat.sub('', outtext)
##     print('3 sub')
##     sys.stdout.flush()
#    outtext = re.sub(r'^.*?\}\}','', outtext)
##     outtext = re.sub(r'^.*?\}\}','', outtext, flags=re.MULTILINE)
##     outtext = tempout.sub('', outtext)
##     print('4 sub')
##     sys.stdout.flush()
#    
#    # links
#    outtext = re.sub(r'\[\[.*?\|', '', outtext)
##     outtext = re.sub(r'\[\[.*?\|', '', outtext, flags=re.MULTILINE)
##     outtext = linkpat.sub('', outtext)
##     print('5 sub')
##     sys.stdout.flush()
    outtext = outtext.replace('[', '')
    outtext = outtext.replace(']', '')
    
    outtext = outtext.replace('{', '')
    outtext = outtext.replace('}', '')
    
    # <ref>
    outtext = outtext.replace('<ref>', '')
    outtext = outtext.replace('</ref>', '')
    
    # titles
    outtext = outtext.replace('=', '')
    
    # categories
    outtext = outtext.replace('Category:', '')
    
    return outtext

In [109]:
# Build editor dictionary with unigrams added
edtext = {ed:'' for ed in talk_eds}

cite_diffdir = '/home/michael/school/research/wp/wp_articles/citediffs/'
# citedir = '/home/michael/school/research/wp/wp_articles/citations/'
# revdir = '/home/michael/school/research/wp/wp_articles/extract/extract_articles/ipc_articles_raw/'

for f in sorted(os.listdir(cite_diffdir))[:2]:
    if not f.startswith('.'):
        print(f)
        with open(os.path.join(cite_diffdir,f)) as file:
            r = csv.DictReader(file)
            data = [row for row in r if row['editor'] in talk_eds and row['add/delete']=='add']
#             diffdata = pd.read_csv()
#             diffdata = diffdata[diffdata['editor'].isin(talk_eds)] # Restrict data to talk editors
#             diffdata = diffdata[diffdata['add/delete']=='add'] # Just adds

101_squadron_(israel)_citediff.csv
2013–14_israeli–palestinian_peace_talks_citediff.csv


In [110]:
# %%timeit -n 1 -r 1

# Try ONE LINE
# Without preprocess: 3.03 s per loop
# With preprocess: 4min 41s without pre-compile, 4min 54s with pre-compile

# Iterate thru citediffs, building editor dict
wdwin = 50

for i, row in enumerate(data):
    if i % 100 == 0: print(i/1000, 'k')
    before = word_tokenize(preprocess(row['before_citation']))
#     if row['editor'] == 'Sepsis II':
#    if '}' in before:
#        debug_here()
#     before = word_tokenize(row['before_citation'])
#     before = preprocess(row['before_citation'])
    edtext[row['editor']] += ' '.join(before[max(len(before)*-1, wdwin*-1):])
    after = word_tokenize(preprocess(row['after_citation']))
#     if '}' in after: debug_here()
#     after = word_tokenize(row['after_citation'])
#     after = preprocess(row['after_citation'])
    edtext[row['editor']] += ' '.join(after[:min(len(after), wdwin)])

print(len([ed for ed in edtext if edtext[ed] != '']))

0.0 k
0.1 k
0.2 k
18


In [111]:
edtext['Sepsis II']

"in 1999 , but never did . Critics also worry that Israel will simply quietly re-arrest the potentially released Palestinians , and state that Israel is using the slow release to hold the negotiations hostage and that the main goal of the release is to bolster Israel 's image .The announcement came after Israel was accused of trying to sabotage the peace talks by approving nearly 1200 new settlement homes , which are on Palestinian land and illegal under international law . Israeli government spokesman Mark Regev dismissed the criticism . See also * Geneva II Middle East peace; he said that what he meant was no Israelis who were `` part of the occupation '' , but that he would n't have a problem with Jews or Israelis coming to Palestine for business or tourism reasons , as long as they were not an occupying force .See also *Peace process in the Israeli–Palestinian conflict * Geneva II Middle East peace conference on Syria References Israeli–Palestinian conflict; he said that what he me

In [25]:
%%timeit -n 1 -r 1

# Without preprocess: 3.03 s per loop
# With preprocess: 4min 41s without pre-compile, 4min 54s with pre-compile

# Iterate thru citediffs, building editor dict
wdwin = 50

for i, row in enumerate(data):
    if i % 100 == 0: print(i/1000, 'k')
    before = word_tokenize(preprocess(row['before_citation']))
#     before = word_tokenize(row['before_citation'])
#     before = preprocess(row['before_citation'])
    edtext[row['editor']] += ' '.join(before[max(len(before)*-1, wdwin*-1):])
    after = word_tokenize(preprocess(row['after_citation']))
#     after = word_tokenize(row['after_citation'])
#     after = preprocess(row['after_citation'])
    edtext[row['editor']] += ' '.join(after[:min(len(after), wdwin)])

print(len(edtext))

0.0 k
0.1 k
0.2 k
5537
1 loop, best of 1: 4min 54s per loop


In [25]:
%%timeit -n 1 -r 1

# TRY AGAIN
# Without preprocess: 3.03 s per loop
# With preprocess: 4min 41s without pre-compile, 4min 54s with pre-compile

# Iterate thru citediffs, building editor dict
wdwin = 50

for i, row in enumerate(data):
    if i % 100 == 0: print(i/1000, 'k')
    before = word_tokenize(preprocess(row['before_citation']))
    matched = re.search(r'(https?:\/\/(?:www\.)?[^ ]*)')
    outtext = re.sub(r'https?:\/\/(?:www\.)?[^ ]*', '', intext, flags=re.MULTILINE)
#     outtext = urlpat.sub('', intext)
#     print('1 sub')
#     sys.stdout.flush()
#     outtext = re.sub(r'[^ ].*?\.html?', '', outtext)
    outtext = re.sub(r'[^ ].*?\.html?', '', outtext, flags=re.MULTILINE)
#     outtext = urlhtml.sub('', outtext)
#     print('2 sub')
#     sys.stdout.flush()
    
    # templates
#     outtext = re.sub(r'\{\{.*?(?:\}\}|$)','', outtext)
    outtext = re.sub(r'\{\{.*?(?:\}\}|$)','', outtext, flags=re.MULTILINE)
#     outtext = temppat.sub('', outtext)
#     print('3 sub')
#     sys.stdout.flush()
#     outtext = re.sub(r'^.*?\}\}','', outtext)
    outtext = re.sub(r'^.*?\}\}','', outtext, flags=re.MULTILINE)
#     outtext = tempout.sub('', outtext)
#     print('4 sub')
#     sys.stdout.flush()
    
    # links
#     outtext = re.sub(r'\[\[.*?\|', '', outtext)
    outtext = re.sub(r'\[\[.*?\|', '', outtext, flags=re.MULTILINE)
#     before = word_tokenize(row['before_citation'])
#     before = preprocess(row['before_citation'])
    edtext[row['editor']] += ' '.join(before[max(len(before)*-1, wdwin*-1):])
    after = word_tokenize(preprocess(row['after_citation']))
#     after = word_tokenize(row['after_citation'])
#     after = preprocess(row['after_citation'])
    edtext[row['editor']] += ' '.join(after[:min(len(after), wdwin)])

print(len(edtext))

0.0 k
0.1 k
0.2 k
5537
1 loop, best of 1: 4min 54s per loop


In [29]:
# %%timeit -n 1 -r 1

# Time-optimized
# Iterate thru citediffs, building editor dict
wdwin = 50

# re sub preprocessing
beforelen = len(data)
#before_data = '\n|||\n'.join(row['before_citation'] for row in data)
#after_data = '\n|||\n'.join(row['after_citation'] for row in data)
before_data = '\n'.join(row['before_citation'] for row in data)
after_data = '\n'.join(row['after_citation'] for row in data)

In [31]:
print('Preprocessing...')
sys.stdout.flush()
before_pp = preprocess(before_data).split('\n')
print('Befores done')
sys.stdout.flush()
after_pp = preprocess(before_data).split('\n')
print('Afters done')
sys.stdout.flush()
assert len(before_pp) == len(after_pp) == beforelen

for i, row in enumerate(data):
    if i % 100 == 0: print(i/1000, 'k')
    before = word_tokenize(before_pp[i])
    edtext[row['editor']] += ' '.join(before[max(len(before)*-1, wdwin*-1):])
    
    after = word_tokenize(after_pp[i])
    edtext[row['editor']] += ' '.join(after[:min(len(after), wdwin)])

print(len(edtext))

Preprocessing...
Befores done
Afters done


AssertionError: 

In [ ]:
def delstring(line, m):
    return line.replace(m.group(), '')

In [11]:
edtext

{'Spesh531': '',
 'Paulistano': '',
 'Hecht': '',
 'Bigshotnews': '',
 'SheffieldSteel': '',
 'Vladko': '',
 'Dr. Manos': '',
 'Karimi cae': '',
 'Yohai.bs': '',
 'Wwb06': '',
 'EETech': '',
 'Kleinecke': '',
 'Cynwolfe': '',
 'Drieakko': '',
 'Light show': '',
 'Yunis': '',
 'Molestash': '',
 'TurtleTurtle': '',
 'Atefrat': '',
 'Oszalał': '',
 'Panyd': '',
 'LemonLion': '',
 'TimeOnHands': '',
 'Leecharleswalker': '',
 'Etothei': '',
 'Eyrian': '',
 'Positively': '',
 'Screen stalker': '',
 'RafaelG': '',
 'Coolnewshere': '',
 'Ashley kennedy3': '',
 'Zginder': '',
 'GRuban': '',
 'Thehistorian10': '',
 'StonedChipmunk': '',
 'Andigisi': '',
 'The Singing Badger': '',
 'Marianian': '',
 'Buspirtraz': '',
 'Aaliyah Stevens': '',
 'Juliancolton': '',
 'Afroghost': '',
 'Jdkag': '',
 'ACEOREVIVED': '',
 'Rodhullandemu': '',
 'Muzher': '',
 'Colipon': '',
 'BozMo': '',
 'E4024': '',
 'Tulli': '',
 'Constanz': '',
 'BrendelSignature': '',
 'Theo F': '',
 'Ahmetyal': '',
 'Lucretius6': '',

In [5]:
outpath = '/home/michael/school/research/wp/wp_articles/editor_citation_window.csv'
with open(outpath, 'w') as f:
    w = csv.writer(f)
    w.writerow(['article_name', 'timestamp', 'editor', 'citation', 'before', 'after',])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [27]:
# Iterate thru citediffs, finding citation windows in text
charwin = 1000

outpath = '/home/michael/school/research/wp/wp_articles/citation_windows/{:s}'.format(f)

with open(outpath, 'w') as f:
    w = csv.writer(f)
    w.writerow(['article_name', 'timestamp', 'editor', 'citation', 'before', 'after',])
    
    for i, row in diffdata.iterrows():
        ts = row['timestamp']
        c = citedata[citedata['timestamp'==ts]['citation']
        ctype = row['citation_type']

        text = revdata[revdata['timestamp']==ts]['text'].values[0]
        ccount = text.count(c)
        print(ccount)
        cidx = text.rfind(c) 
        after_idx = cidx+len(c)+1

        before = text[max(cidx-charwin,0):cidx]
        after = text[after_idx:min(after_idx+charwin,len(text))]
    #     print(before)
    #     print()
    #     print(preprocess(before))
    # #     print(len(word_tokenize(preprocess(before))))
    #     print()
    #     print(after)
    #     print()
    #     print(preprocess(after))
    # #     print(len(word_tokenize(preprocess(after))))

        if i > 100: break

1
 set to begin after Israel agrees to free 104 Palestinian prisoners}}</ref>  and will then move to the [[King David Hotel bombing|King David Hotel in Jerusalem]][http://www.timesofisrael.com/us-peace-envoy-to-come-to-israel-next-week/] and then to Hebron.  ==Prisoner release== In July 2013, Israel announced as a "good will jesture" that it would be releasing 104 Palestinian prisoners, all of whom have been in Israeli jails since before the 1993 Oslo Accords. Israel then stated that it would be releasing the prisoners slowing throughout the negotiations rather than before the negotiations.   Commenters have however pointed out that Israel had already promised to release these same 104 Palestinians, back in 1999, but never did. Critics also worry that Israel will simply quietly re-arrest the potentially released Palestinians, and state that Israel is using the slow release to hold the negotiations hostage and that the main goal of the release is to bolster Israel's image.<ref>http://ww